## 01_SolicitudOrdenAtlasConTrazado
Autor: Carlos Wong <br/>
email: carlosawongca@gmail.com <br/>
Fecha: 2023_04_08 <br/>
Objetivo: Edición automática email <br/>
Entrada: 'content.xlsx' <br/>
Salida: - <br/>
Propósito: Prueba edición automática de emails <br/>

In [1]:
# librerías estándar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import win32com.client as win32
from tabulate import tabulate

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Ruta del archivo Excel
ruta_archivo = r'.\trazado_STa.xlsx'

# Leer el archivo Excel
excel_file = pd.ExcelFile(ruta_archivo)

In [4]:
# Crear un diccionario vacío para almacenar los dataframes de cada hoja
dataframes_por_hoja = {}
# Iterar a través de cada hoja del archivo Excel
for nombre_hoja in excel_file.sheet_names:
    
    # Leer la hoja y seleccionar las columnas a partir de la tercera columna
    hoja = pd.read_excel(excel_file, sheet_name=nombre_hoja, header=1)
    hoja = hoja.iloc[:, 2:]
    
    # Eliminar filas con valores nulos
    hoja = hoja.dropna(subset=['Codificación Común'])
    
    # Agregar el dataframe a nuestro diccionario con el nombre de la hoja como clave
    dataframes_por_hoja[nombre_hoja] = hoja

In [156]:
def emailer(asignaciones):
    df = pd.read_excel(r'.\Seguimiento_CCVV_HUAWEI_S.TA_RI.xlsx', header=1, sheet_name='RI_FO')
    cuerpo_email = "Buenas, <br/><br/> La central de SEVILLA TABLADILLA (4110008 – S.TA) se encuentra en fase de repliegue tras ser <b>CERRADA POR FARO</b>. <br/><br/> Solicitamos la creación de las órdenes de REASIGNACIÓN INTERNA para los siguientes circuitos: <br/><br/>"
    contador = 1
    for asigna in asignaciones:
        cuerpo_email += f"<b>RI {contador}</b> <br/><br/>"

        contador += 1
        row = df.loc[df['ASIGNACION'] == asigna]
        if len(row) == 0:
            print(f"No se encontró la asignación {asigna} en la tabla")
            continue
        
        admntvo = int(row['ADMINISTRATIVO'].values[0])
        Cr = int(row['Cr'].values[0])
        Ptro = int(row['Ptro'].values[0])
        Arbol = row['Arbol'].values[0]
        FacActual = row['FacActual'].values[0]
        FacNueva = row['FacNueva'].values[0]
        Fusion = row['Fusion'].values[0]
        CorteEnCaliente = row['CorteEnCaliente'].values[0]
        
        dataframe = dataframes_por_hoja[str(admntvo)]
        datos = dataframe.values.tolist()
        longitudes_maximas = [max([len(str(row[i])) for row in datos]) for i in range(len(dataframe.columns))]
        tabla = tabulate(datos, tablefmt="plain", maxcolwidths=longitudes_maximas)
        tabla_html = tabulate(datos, tablefmt="html") # Convertir la tabla a formato HTML
        cuerpo_email += f"1. Ordenes CON cambio de terminal Z <br/>{asigna}{'' if pd.isna(admntvo) else f'  -  {admntvo}'} <br/><br/>"
        cuerpo_email += f"\nTRAZADO ACTUAL:<br/>- Facilidad actual: {FacActual} (se requiere liberar los puertos de ésta facilidad)<br/><br/>"
        cuerpo_email += f"{tabla_html}<br/>" # Agregar la tabla al cuerpo
        cuerpo_email += f"TRAZADO NUEVO :<br/>- Facilidad nueva: {FacNueva} a PTRO {Ptro}<br/>- Fusión en CR {Cr}<br/>- Fusión: {Fusion}<br/>- Corte en Caliente: {CorteEnCaliente}<br/><br/>2. Ordenes CON cambio de terminal Z <br/><br/><br/>"
        
    cuerpo_email += "Estas órdenes deben ser creadas con alternativa de jalonamiento “FARH”. En caso de no ser posible, por favor, jalonar con alternativa genérica.<br/><br/>Adicionalmente, las órdenes deben ser marcadas como grupo de órdenes con la denominación HSTA – #TOR# FABRICA REPLIEGUES HUAWEI.<br/><br/>Un saludo,<br/><br/>Carlos Wong"
    
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.Subject = f"FRHUA - Solicitud creación RI SEVILLA TABLADILLA (4110008 - S.TA)"
    mail.HTMLBody = cuerpo_email
    mail.Display(True)  # muestra la previsualización del correo

In [157]:
emailer(['S.RD0016FO03 /S.TA ETH 1/DP /X0ETHD/40984559/CONSTAECA', 'S.TA0300 /S.NE ETH 1/DP /U3OENM/10766067/LINCETELE'])